In [9]:
def rank(r):
    return r.split("\\")[1].split("-")[0]

In [10]:
import pandas as pd
import glob
csv_f = pd.Series(glob.glob("./YT/*.csv"))
df = pd.DataFrame({"rank":csv_f.apply(rank), "path":csv_f})
df1 = df[[int(r) // 10 == 0 for r in df["rank"]]]
df2 = df.drop(df1.index)
df = pd.concat([df1, df2]).reset_index(drop=True)
df

,rank,path
0,1,./YT\1-蔡阿嘎.csv
1,2,./YT\2-這群人 TGOP.csv
2,4,./YT\4-王宏哲教養、育兒寶典.csv
3,5,./YT\5-那對夫妻.csv
4,7,./YT\7-黃阿瑪的後宮生活.csv
5,8,./YT\8-阿滴英文.csv
6,9,./YT\9-486先生.csv
7,10,./YT\10-眾量級CROWD.csv
8,11,./YT\11-谷阿莫 AmoGood.csv
9,12,./YT\12-蔡桃貴.csv


In [11]:
import os

name_list = []
video_num_list = []

for f in df["path"]:
    df3 = pd.read_csv(f)
#     name = f.split("\\")[1].split("-")[1].split(".")[0]
    name = os.path.splitext(f)[0].split("\\")[1].split("-")[1]
    name_list.append(name)
    video_num = df3.shape[0]
    video_num_list.append(video_num)
df["name"] = name_list
df["video_num"] = video_num_list
df

,rank,path,name,video_num
0,1,./YT\1-蔡阿嘎.csv,蔡阿嘎,8094
1,2,./YT\2-這群人 TGOP.csv,這群人 TGOP,942
2,4,./YT\4-王宏哲教養、育兒寶典.csv,王宏哲教養、育兒寶典,1832
3,5,./YT\5-那對夫妻.csv,那對夫妻,16636
4,7,./YT\7-黃阿瑪的後宮生活.csv,黃阿瑪的後宮生活,1817
5,8,./YT\8-阿滴英文.csv,阿滴英文,3940
6,9,./YT\9-486先生.csv,486先生,475
7,10,./YT\10-眾量級CROWD.csv,眾量級CROWD,3916
8,11,./YT\11-谷阿莫 AmoGood.csv,谷阿莫 AmoGood,14515
9,12,./YT\12-蔡桃貴.csv,蔡桃貴,908


In [12]:
df_all = pd.read_csv("ALL.csv")
df_class = df_all[["名次", "分類"]]
df["分類"] = df_class[[str(r) in list(df["rank"]) for r in df_class["名次"]]].reset_index(drop=True)["分類"]
df = df.drop(["path"], axis=1)
df = df[["分類", "rank", "name", "video_num"]]
df
# df.to_csv("IG_num.csv", index=False)

,分類,rank,name,video_num
0,生活,1,蔡阿嘎,8094
1,搞笑,2,這群人 TGOP,942
2,親子,4,王宏哲教養、育兒寶典,1832
3,親子,5,那對夫妻,16636
4,寵物,7,黃阿瑪的後宮生活,1817
5,教育,8,阿滴英文,3940
6,開箱,9,486先生,475
7,生活,10,眾量級CROWD,3916
8,電影,11,谷阿莫 AmoGood,14515
9,親子,12,蔡桃貴,908


In [15]:
classes = list(set(df["分類"]))
num_list = []
image_num_list = []
video_num_list = []
for c in classes:
    df_sub = df[[str(r) in c for r in df["分類"]]].reset_index(drop=True)
    num_list.append(df_sub.shape[0])
    video_num_list.append(df_sub["video_num"].sum())
df_final = pd.DataFrame([classes, num_list, video_num_list]).T
df_final.columns = ["分類", "人數", "影片數量"]
df_final = df_final.sort_values(by=["影片數量"], ascending=False).reset_index(drop=True)
df_final

# df_final.to_csv("IG_num_class.csv")

,分類,人數,影片數量
0,親子,3,19376
1,搞笑,2,15556
2,電影,1,14515
3,生活,2,12010
4,教育,1,3940
5,寵物,1,1817
6,開箱,1,475


In [16]:
classes = list(df_final["分類"])
df_sub_all = pd.DataFrame()
for c in classes:
    df_sub = df[[str(r) in c for r in df["分類"]]].reset_index(drop=True)
    df_sub["video_num_percent"] = round(df_sub["video_num"] / df_sub["video_num"].sum() * 100, 2)
    df_sub_all = pd.concat([df_sub_all, df_sub])
pd.set_option('display.max_rows', 100)
df_sub_all = df_sub_all[["分類", "rank", "name", "video_num", "video_num_percent"]].reset_index(drop=True)
df_sub_all

# df_sub_all.to_csv("IG_num_percent.csv")

,分類,rank,name,video_num,video_num_percent
0,親子,4,王宏哲教養、育兒寶典,1832,9.45
1,親子,5,那對夫妻,16636,85.86
2,親子,12,蔡桃貴,908,4.69
3,搞笑,2,這群人 TGOP,942,6.06
4,搞笑,13,在不瘋狂就等死,14614,93.94
5,電影,11,谷阿莫 AmoGood,14515,100.00
6,生活,1,蔡阿嘎,8094,67.39
7,生活,10,眾量級CROWD,3916,32.61
8,教育,8,阿滴英文,3940,100.00
9,寵物,7,黃阿瑪的後宮生活,1817,100.00


生活 26 19794 5751
插畫 18 11088 3092
美妝 4 6901 3428
搞笑 9 6663 3925
親子 4 5958 2818
寵物 4 3171 755
音樂 3 1827 708